# Analyzing Results - Anomalies Detection with Gender Approach

## Loading Initial Datasets

In [0]:
import pandas as pd
import numpy as np

In [0]:
#150760  rows
total_dataset = pd.read_csv('meryem_dataset.csv',error_bad_lines=False,encoding='utf-8', sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
total_dataset.head()

,Unnamed: 0,xd_id,user_id,product,nb_views,product_id,product_gender,product_gender_proba,product_Gid,ponderation,user_id_sex_MALE,user_id_sex_FEMALE,user_id_sex_UNISEX,validation_interval_MALE_b,validation_interval_FEMALE_b,validation_interval_UNISEX_b,validation_interval_MALE_t,validation_interval_FEMALE_t,validation_interval_UNISEX_t
0,0,9af1350f-ed15-3e33-a601-1e7f32ac245f,10039a67-1a45-458f-9a9b-995fef41dd0e,"[""13927_286566958_4548658863070807158"",""FEMALE...",1,13927_286566958_4548658863070807158,FEMALE,0.749126,201,0.749126,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,1,9af1350f-ed15-3e33-a601-1e7f32ac245f,10039a67-1a45-458f-9a9b-995fef41dd0e,"[""13927_661180985_9061753854348244413"",""FEMALE...",2,13927_661180985_9061753854348244413,FEMALE,0.749915,201,1.499830,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2,9af1350f-ed15-3e33-a601-1e7f32ac245f,10039a67-1a45-458f-9a9b-995fef41dd0e,"[""13927_623500964_1503367560184714452"",""FEMALE...",1,13927_623500964_1503367560184714452,FEMALE,0.749249,5193,0.749249,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,3,9af1350f-ed15-3e33-a601-1e7f32ac245f,10039a67-1a45-458f-9a9b-995fef41dd0e,"[""13927_1080215607_-7864946025625050524"",""FEMA...",1,13927_1080215607_-7864946025625050524,FEMALE,0.749891,201,0.749891,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,4,9af1350f-ed15-3e33-a601-1e7f32ac245f,10039a67-1a45-458f-9a9b-995fef41dd0e,"[""13927_1053484283_-8223979396819036224"",""FEMA...",1,13927_1053484283_-8223979396819036224,FEMALE,0.743948,201,0.743948,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [0]:
# 34601 xd_id
perf_metrics = pd.read_csv('gdrive/Team Drives/Data Science - XDevice - OMA Recherche/extract_perf_metrics.csv', sep='\t')

In [0]:
#33910 xd_id
total_dataset_xd_id_group = pd.DataFrame(total_dataset.groupby('xd_id').user_id.nunique())
total_dataset_xd_id_group = total_dataset_xd_id_group.reset_index()

In [0]:
total_dataset_xd_id_perf = total_dataset_xd_id_group.merge(perf_metrics, on = 'xd_id')

In [0]:
results_clicks = pd.DataFrame(total_dataset_xd_id_perf.groupby('user_id').nb_clicks_desktop.sum())
results_displays = pd.DataFrame(total_dataset_xd_id_perf.groupby('user_id').nb_displays_desktop.sum())

In [0]:
results_clicks.columns = ['nb_clicks_with_anomalies']
results_displays.columns = ['nb_displays_with_anomalies']
results = results_clicks
results['nb_displays_with_anomalies'] = results_displays['nb_displays_with_anomalies']

In [0]:
results['CTR - with anomalies'] = results['nb_clicks_with_anomalies']/results['nb_displays_with_anomalies']

## Removing Anomalies

In [0]:
# 5783 xd_id are anomalies
anomalies = pd.read_csv('xid_anomaly.txt', header = None)
anomalies.columns = ['xd_id']
anomalies_list = anomalies['xd_id'].tolist()

In [0]:
#Remove Anomalies : 28117 xd_id are left
total_dataset_xd_id_perf_without_anomalies = total_dataset_xd_id_perf[~total_dataset_xd_id_perf.xd_id.isin(anomalies_list)]

In [0]:
results['nb_clicks_without_anomalies'] = total_dataset_xd_id_perf_without_anomalies.groupby('user_id').nb_clicks_desktop.sum()
results['nb_displays_without_anomalies'] = total_dataset_xd_id_perf_without_anomalies.groupby('user_id').nb_displays_desktop.sum()

In [0]:
results['CTR - without anomalies'] = results['nb_clicks_without_anomalies']/results['nb_displays_without_anomalies']

In [0]:
results = results.reset_index()

In [0]:
results = results.drop(['nb_clicks_with_anomalies', 'nb_displays_with_anomalies',
       'nb_clicks_without_anomalies','nb_displays_without_anomalies'], axis=1)

In [0]:
results.columns = ['Cluster size', 'CTR - with anomalies', 'CTR - without anomalies']

In [0]:
results

,Cluster size,CTR - with anomalies,CTR - without anomalies
0,1,0.004392,0.004392
1,2,0.004574,0.004574
2,3,0.004546,0.004546
3,4,0.004971,0.004971
4,5,0.004944,0.004944
5,6,0.004985,0.004855
6,7,0.005402,0.005069
7,8,0.005010,0.004215
8,9,0.005496,0.004498
9,10,0.005236,0.004334


In [0]:
nbre_xd_id_per_cluster_size = pd.DataFrame(total_dataset_xd_id_group.groupby('user_id').xd_id.nunique())
nbre_xd_id_per_cluster_size['Number of xd_id - without anomalies'] = total_dataset_xd_id_perf_without_anomalies.groupby('user_id').xd_id.nunique()
nbre_xd_id_per_cluster_size['Number of xd_id - without anomalies'].fillna(0, inplace=True)
nbre_xd_id_per_cluster_size = nbre_xd_id_per_cluster_size.reset_index()
nbre_xd_id_per_cluster_size.columns = ['Cluster size', 'Number of xd_id - Total', 'Number of xd_id - without anomalies']
nbre_xd_id_per_cluster_size['Number of anomalies'] = nbre_xd_id_per_cluster_size['Number of xd_id - Total'] - nbre_xd_id_per_cluster_size['Number of xd_id - without anomalies']
nbre_xd_id_per_cluster_size['Percentage of anomalies'] = nbre_xd_id_per_cluster_size['Number of anomalies']/nbre_xd_id_per_cluster_size['Number of xd_id - Total']

In [0]:
nbre_xd_id_per_cluster_size

,Cluster size,Number of xd_id - Total,Number of xd_id - without anomalies,Number of anomalies,Percentage of anomalies
0,1,2566,2556.0,10.0,0.003897
1,2,4754,4754.0,0.0,0.000000
2,3,6052,6052.0,0.0,0.000000
3,4,6050,6050.0,0.0,0.000000
4,5,4947,4947.0,0.0,0.000000
5,6,3535,1883.0,1652.0,0.467327
6,7,2366,951.0,1415.0,0.598056
7,8,1557,464.0,1093.0,0.701991
8,9,877,233.0,644.0,0.734322
9,10,533,119.0,414.0,0.776735


In [0]:
results = results.merge(nbre_xd_id_per_cluster_size, on='Cluster size')

In [0]:
results['Amélioration du CTR'] = (results['CTR - without anomalies'] - results['CTR - with anomalies'])/results['CTR - with anomalies']

In [0]:
results

,Cluster size,CTR - with anomalies,CTR - without anomalies,Number of xd_id - Total,Number of xd_id - without anomalies,Number of anomalies,Percentage of anomalies,Amélioration du CTR
0,1,0.004392,0.004392,2566,2556.0,10.0,0.003897,0.000000
1,2,0.004574,0.004574,4754,4754.0,0.0,0.000000,0.000000
2,3,0.004546,0.004546,6052,6052.0,0.0,0.000000,0.000000
3,4,0.004971,0.004971,6050,6050.0,0.0,0.000000,0.000000
4,5,0.004944,0.004944,4947,4947.0,0.0,0.000000,0.000000
5,6,0.004985,0.004855,3535,1883.0,1652.0,0.467327,-0.025999
6,7,0.005402,0.005069,2366,951.0,1415.0,0.598056,-0.061634
7,8,0.005010,0.004215,1557,464.0,1093.0,0.701991,-0.158554
8,9,0.005496,0.004498,877,233.0,644.0,0.734322,-0.181683
9,10,0.005236,0.004334,533,119.0,414.0,0.776735,-0.172252
